In [2]:
import mysql.connector as mysql
from mysql.connector import Error
import os
from dotenv import load_dotenv

# Script to connect

In [3]:
# Database credentials (replace with your details)
load_dotenv()

hostname=os.getenv('hostname')
port=3306
username='tkirk'
password=os.getenv('password')
ssl_ca = os.getenv('ssl_cert')  # Required for Azure MySQL SSL connection
database = 'wildfire_housing'

try:
    # Establish connection
    conn = mysql.connect(
        host=hostname,
        user=username,
        password=password,
        ssl_ca=ssl_ca,  # Secure connection
        database = database
    )

    if conn.is_connected():
        print("✅ Successfully connected to MySQL database")

except Error as e:
    print(f"❌ Error: {e}")



✅ Successfully connected to MySQL database


# Sample Query

In [ ]:
#cursor = conn.cursor()

query ="""
SELECT h.county_name, 
    YEAR(h.assessment_date) AS Year, 
    COUNT(w.fire_id) AS count,
    SUM(w.fire_size) AS total_burned
FROM housing h
INNER JOIN wildfire w ON h.county_id = w.county_id AND YEAR(w.discovery_date) = YEAR(h.assessment_date)
WHERE YEAR(h.assessment_date) = 2015
GROUP BY h.county_name, YEAR(h.assessment_date)
LIMIT 100;
"""

df = pd.read_sql(query, conn)

C:\Users\Taylor\AppData\Local\Temp\ipykernel_15460\3279630705.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [13]:
df.head()

,county_name,Year,count,total_burned
0,Alachua County,2015,15930,219060.00
1,Alameda County,2015,36108,496536.00
2,Angelina County,2015,20160,2999006.40
3,Appling County,2015,1062,14604.00
4,Atascosa County,2015,2460,28805.04


# Script to create schema

In [14]:
cursor = conn.cursor()

try:
    cursor.execute('CREATE DATABASE IF NOT EXISTS wildfire_housing')
    print('Database create or already exists')

except mysql.Error as err:
    print(f"Error creating database")

conn.database = database
    
with open('schema.sql', 'r') as schema:
    sql_script = schema.read()

try:

    for statement in sql_script.split(";\n"):
        statement = statement.strip()
        if statement:
            cursor.execute(statement)
except mysql.Error as err:
    print(f"Error executing statement: {statement}")
    print(f"MySQL Error: {err}")

conn.commit()
cursor.close()
conn.close()

print('Database schema created successfully')


Database create or already exists
Database schema created successfully
